In [1]:
import os
import glob
import ray
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import functools as ft
import operator as op
from module import JP_Parser, Example

In [2]:
os.chdir('/Users/sheg/Documents/projects/anki-flashcards/japanese/data')

# Задание порядка карточек

In [22]:
vocab = pd.read_pickle('vocab.pkl')
kanji = pd.read_pickle('kanji.pkl')
radic = pd.read_pickle('radic.pkl')

In [4]:
from enum import Enum

In [5]:
class card_type(Enum):
    radic = 0
    kanji = 1
    vocab = 2

In [6]:
def find_kanji(text, df=kanji, *, drop=False):
    
    chars = list(text)
    chars = pd.Series(chars)
    true_kanji = chars[ chars.isin(df.kanji) ]
    
    mask = df.kanji.isin(true_kanji)
    result = df[mask]
    
    if drop:
        df.drop(result.index, inplace=True)
    
    return result

In [39]:
_kanji = kanji.copy()
_radic = radic.copy()

used_radic = []
cards_order = []

# первые 60 слов
for word_tuple in vocab.iloc[:60].itertuples():
    
    kanji_str = word_tuple.kanji
    
    if type(kanji_str) == str:
        kanji_found = find_kanji(kanji_str, df=_kanji, drop=True)
        
        for rads, kanji_ in kanji_found[['radic', 'kanji']].itertuples(index=False):
            for rad_n in rads:
                radic_str = radic.loc[rad_n].radic
                if radic_str == kanji_:
                    continue
                if rad_n in used_radic:
                    continue
                cards_order.append( (card_type.radic, rad_n) )
            
        kanji_indices = kanji_found.index
        cards_order.extend( zip([card_type.kanji] * len(kanji_indices), kanji_indices) )        
    
    if kanji_str not in kanji_found.kanji.values:
        cards_order.append( (card_type.vocab, word_tuple.Index) )

In [40]:
len(cards_order)

139